In [1]:
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index(host=os.getenv("PINECONE_HOST"))

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(os.getenv("MODEL_NAME"))
model = AutoModelForCausalLM.from_pretrained(
    os.getenv("MODEL_NAME"), device_map="auto", dtype="auto"
)

In [3]:
def answer_query_with_context(query: str, top_k: int = 3, max_tokens: int = 500):
    # Pinecone search
    results = index.search(
        namespace=os.getenv("PINECONE_NAMESPACE"),
        query={"inputs": {"text": query}, "top_k": top_k},
        fields=["text", "source", "category"]
    )
    
    # Extracting text from hits
    hits = results["result"]["hits"]
    context = "\n".join([hit["fields"]["text"] for hit in hits if "fields" in hit and "text" in hit["fields"]])
    if not context:
        context = "No relevant information found in the knowledge base."
    
    # Prompt
    prompt = (
        f"Answer the question using the context below. "
        f"Answer as accurately as possible based on the context.\n\n"
        f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    )
    
    # Tokenization and generation
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_tokens)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return answer

In [4]:
user_query = "Give me the title of your favorite research paper and why"
answer = answer_query_with_context(user_query)
print(answer)

Answer the question using the context below. Answer as accurately as possible based on the context.

Context:
This is one of my research paper. This research paper is for our course subject Artificial Intelligence and Data Science. This research paper won us the AI-Deas 2025 Cordillera Administrative Region (CAR) Leg. It is by far my favorite research paper as it is very novel and I'm the principal author of this paper.
The TITLE of this research paper is: MFCC-Chroma Fusion using a Dual-Channel CNN-LSTM and Demographic-Aware Random Forest for Lung Sound Disease Classification
The ABSTRACT of this research paper is: Pulmonary diseases continue to pose significant diagnostic challenges. Auscultation, a common method for lung sound analysis, remains subjective and prone to inconsistency. This study proposes two complementary approaches for improving lung disease detection: (1) a dual-channel CNN-LSTM model using MFCC-Chroma fusion for automated lung sound classification, and (2) a Random